In [23]:
import numpy as np
import pandas as pd

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from pyomo.environ import *
from highspy import Highs
from pprint import pprint

products_list = ['A', 'B', 'C'] # Monitor, TV, Refrigerator
time_periods = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

required_hours_per_unit = {'A': 2, 'B': 3, 'C': 5}

initial_inventory = {'A': 200, 'B': 200, 'C': 200}
space_factors = {'A': 0.2, 'B': 0.5, 'C': 1.0}

over_time_fraction = 0.25 # 25% of Regular hours are allowed for over time

scenes = ['low', 'med', 'high']
probs = {
    'low': 0.25,
    'med': 0.5,
    'high': 0.25
}

prod_pd = pd.read_csv('product_data.csv', index_col='pt_index')


# 3. Model Construction

In [5]:
model = ConcreteModel()

model.P = Set(initialize=products_list)
model.T = Set(initialize=time_periods)
model.S = Set(initialize=scenes)

model.pt_index = Set(initialize=prod_pd.index)

## 3.1 Parameters

In [6]:
model.unit_production_cost = Param(model.pt_index, initialize=prod_pd['unit_production_cost'].to_dict(), within=NonNegativeReals)
model.unit_holding_cost = Param(model.pt_index, initialize=prod_pd['unit_holding_cost'].to_dict(), within=NonNegativeReals)
model.unit_backorder_cost = Param(model.pt_index, initialize=prod_pd['unit_backorder_cost'].to_dict(), within=NonNegativeReals)
model.unit_reg_labor_cost = Param(model.pt_index, initialize=prod_pd['unit_reg_labor_cost'].to_dict(), within=NonNegativeReals)
model.unit_ot_labor_cost = Param(model.pt_index, initialize=prod_pd['unit_ot_labor_cost'].to_dict(), within=NonNegativeReals)
model.hiring_cost = Param(model.pt_index, initialize=prod_pd['hiring_cost'].to_dict(), within=NonNegativeReals)
model.firing_cost = Param(model.pt_index, initialize=prod_pd['firing_cost'].to_dict(), within=NonNegativeReals)


model.required_hours_per_unit = Param(model.P, initialize=required_hours_per_unit, within=NonNegativeIntegers)
model.max_hours_per_worker = Param(initialize=40) # max regular working hours per week
model.over_time_fraction  = Param(initialize=0.25)


model.initial_inventory = Param(model.P, initialize=initial_inventory)
model.initial_workers = Param(
    model.P,
    initialize={'A': 60, 'B': 70, 'C': 80},
    within=NonNegativeIntegers
)


def demand_init(m, p, t, s):
    return getattr(prod_pd.loc[f'{p}_{t}'], f'{s}_demand')

model.demand = Param(model.P, model.T, model.S, initialize=demand_init, within=NonNegativeIntegers)


model.probs = Param(model.S, initialize=probs)

model.space_factor = Param(model.P, initialize=space_factors)

## 3.2 Decision Variables

In [7]:
## Stage 1 Variables
model.R = Var(model.P, model.T, within=NonNegativeReals) # Regular working hours
model.O = Var(model.P, model.T, within=NonNegativeReals) # Over time working hours


model.W = Var(model.P, model.T, within=NonNegativeReals) # No. of Workers
model.H = Var(model.P, model.T, within=NonNegativeReals) # Hiring additional no. of workers
model.F = Var(model.P, model.T, within=NonNegativeReals) # Firing available no. of workers
model.X = Var(model.P, model.T, within=NonNegativeIntegers) # Production


## Stage 2 Variables

model.I_regular = Var(model.P, model.T, model.S, within=NonNegativeIntegers)  # In-house warehouse
model.I_overflow = Var(model.P, model.T, model.S, within=NonNegativeIntegers)  # Rented storage
model.B = Var(model.P, model.T, model.S, within=NonNegativeIntegers)  # Backorders


## 3.3 Constraints

### 3.3.1 Workforce Constraints

In [8]:
## Workforce Balance

def workforce_balance_rule(m, p, t):
    if not t==m.T.first():
        return m.W[p, t] == m.W[p, m.T.prev(t)] + m.H[p, t] - m.F[p, t]
    else:
        return m.W[p, t] == m.initial_workers[p] + m.H[p, t] - m.F[p, t]
    
model.workforce = Constraint(model.P, model.T, rule=workforce_balance_rule)

In [9]:
## Labour hours

def regular_hours_rule(m, p, t):
    return m.R[p, t] <= m.W[p, t] * m.max_hours_per_worker

model.regular_hours = Constraint(model.P, model.T, rule=regular_hours_rule)

In [10]:
## Over Time Limit

def overtime_limit_rule(m, p, t):
    return m.O[p, t] <= m.over_time_fraction * m.R[p, t]

model.overtime_limit = Constraint(model.P, model.T, rule=overtime_limit_rule)

### 3.3.2 Production and Inventory Constraints

In [11]:

total_max_regular = 1000   # Total "spots" in the main warehouse
total_max_overflow = 500   # Total "spots" in the rented warehouse

max_backorder_fraction = 0.4 # maximum backorders allowed is set as 50% to limit them, keeping in mind the backorder costs 

In [12]:
def global_regular_inv_cap_rule(m, t, s):
    # Sum of (Units * SpacePerUnit) for all products must be <= Total Space
    return sum(m.I_regular[p, t, s] * m.space_factor[p] for p in m.P) <= total_max_regular

model.global_regular_inv_cap = Constraint(model.T, model.S, rule=global_regular_inv_cap_rule)

In [13]:
def global_overflow_inv_cap_rule(m, t, s):
    return sum(m.I_overflow[p, t, s] * m.space_factor[p] for p in m.P) <= total_max_overflow

model.global_overflow_inv_cap = Constraint(model.T, model.S, rule=global_overflow_inv_cap_rule)

In [14]:
## Production capability

def production_capability_rule(m, p, t):
    return m.required_hours_per_unit[p] * m.X[p, t] <= m.R[p, t] + m.O[p, t]

model.production_capability = Constraint(model.P, model.T, rule=production_capability_rule)

In [15]:
## Inventory capacity

def inventory_capacity_rule(m, p, t, s):
    production = m.X[p, t]
    demand = m.demand[p, t, s]
    current_inventory = m.I_regular[p, t, s] + m.I_overflow[p, t, s]
    current_backorder = m.B[p, t, s]
    
    if t == m.T.first():
        prev_inventory = m.initial_inventory[p]
        prev_backorder = 0
    else:
        prev_inventory = m.I_regular[p, m.T.prev(t), s] + m.I_overflow[p, m.T.prev(t), s]
        prev_backorder = m.B[p, m.T.prev(t), s]
    
    return prev_inventory - prev_backorder + production - demand == current_inventory - current_backorder

model.inventory_capacity_rule = Constraint(model.P, model.T, model.S, rule=inventory_capacity_rule)

In [16]:
## Backorder Limit Rule

def back_limit_rule(m, p, t, s):
    return m.B[p, t, s] <= max_backorder_fraction * m.demand[p, t, s]


model.back_limit = Constraint(model.P, model.T, model.S, rule=back_limit_rule)

## 3.5 Objective Function

In [17]:
## Objective Function

def objective_rule(m):
    stage1_costs = sum(
        m.unit_reg_labor_cost[f'{p}_{t}'] * m.R[p, t] +
        m.unit_ot_labor_cost[f'{p}_{t}'] * m.O[p, t] +
        m.hiring_cost[f'{p}_{t}'] * m.H[p, t] +
        m.firing_cost[f'{p}_{t}'] * m.F[p, t] +
        m.unit_production_cost[f'{p}_{t}'] * m.X[p, t]
        for p in m.P for t in m.T
    )

    stage2_costs = sum(
        m.probs[s] * (
            m.unit_holding_cost[f'{p}_{t}'] * m.I_regular[p, t, s] +  # Regular storage cost
            3 * m.unit_holding_cost[f'{p}_{t}'] * m.I_overflow[p, t, s] +  # 3x cost for overflow
            m.unit_backorder_cost[f'{p}_{t}'] * m.B[p, t, s]
        )
        for p in m.P for t in m.T for s in m.S
    )

    return stage1_costs + stage2_costs

model.obj = Objective(rule=objective_rule, sense=minimize)

## 3.6 Sovling the Problem

In [18]:
solver = SolverFactory("highs")
results = solver.solve(model, tee=True, load_solutions=False)

print("\n" + "="*60)
print("SOLUTION STATUS:", results.solver.termination_condition)
print("="*60)

if str(results.solver.termination_condition) == 'optimal':
    model.solutions.load_from(results)
    print(f"\n✅ OPTIMAL SOLUTION FOUND")
    print(f"Total Cost: ${value(model.obj):,.2f}")
    
    # Quick sanity checks
    print("\n--- SOLUTION SUMMARY ---")
    total_production = sum(value(model.X[p, t]) for p in model.P for t in model.T)
    total_hiring = sum(value(model.H[p, t]) for p in model.P for t in model.T)
    total_firing = sum(value(model.F[p, t]) for p in model.P for t in model.T)
    
    print(f"Total Production: {total_production:,.0f} units")
    print(f"Total Hiring: {total_hiring:,.0f} workers")
    print(f"Total Firing: {total_firing:,.0f} workers")
    
else:
    print(f"\n❌ MODEL INFEASIBLE")
    print("Try increasing backorder_fraction to 0.5 or initial_workers")

Running HiGHS 1.12.0 (git hash: n/a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 432 rows; 540 cols; 1446 nonzeros; 360 integer variables (0 binary)
Coefficient ranges:
  Matrix  [2e-01, 4e+01]
  Cost    [5e-01, 1e+03]
  Bound   [0e+00, 0e+00]
  RHS     [4e+00, 1e+03]
Presolving model
324 rows, 537 cols, 1335 nonzeros  0s
295 rows, 469 cols, 1273 nonzeros  0s
Presolve reductions: rows 295(-137); columns 469(-71); nonzeros 1273(-173) 

Solving MIP model with:
   295 rows
   469 cols (0 binary, 360 integer, 0 implied int., 109 continuous, 0 domain fixed)
   1273 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tre

# 4. Analysis

## Executive Summary

In [20]:
# Prices
prices = {'A': 150, 'B': 500, 'C': 650}

# Containers for aggregation
total_rev = np.zeros(len(time_periods))
total_cost = np.zeros(len(time_periods))

product_data = {}

for p in products_list:
    p_rev = []
    p_cost = []
    
    for t_idx, t in enumerate(model.T):
        # Expected Revenue = Price * E[Demand met in this period]
        # Demand met = Demand[s] - Backorders[s]
        exp_sold = sum(model.probs[s] * (value(model.demand[p, t, s]) - value(model.B[p, t, s])) for s in model.S)
        rev = prices[p] * exp_sold
        
        # Expected Costs (Stage 1 + Expected Stage 2)
        s1_cost = value(
            model.unit_reg_labor_cost[f'{p}_{t}'] * model.R[p, t] + 
            model.unit_ot_labor_cost[f'{p}_{t}'] * model.O[p, t] +
            model.hiring_cost[f'{p}_{t}'] * model.H[p, t] + 
            model.firing_cost[f'{p}_{t}'] * model.F[p, t] +
            model.unit_production_cost[f'{p}_{t}'] * model.X[p, t]
        )
        s2_cost = sum(model.probs[s] * (
            value(model.unit_holding_cost[f'{p}_{t}']) * value(model.I_regular[p, t, s]) +
            3 * value(model.unit_holding_cost[f'{p}_{t}']) * value(model.I_overflow[p, t, s]) +
            value(model.unit_backorder_cost[f'{p}_{t}']) * value(model.B[p, t, s])
        ) for s in model.S)
        
        cost = s1_cost + s2_cost
        
        p_rev.append(rev)
        p_cost.append(cost)
        total_rev[t_idx] += rev
        total_cost[t_idx] += cost
        
    product_data[p] = {'rev': p_rev, 'cost': p_cost, 'profit': [r - c for r, c in zip(p_rev, p_cost)]}

# Global Profit
total_profit = total_rev - total_cost



# Reuse the data from the previous step
fin_df = pd.DataFrame({
    'Week': time_periods,
    'Total Revenue': total_rev,
    'Total Cost': total_cost,
    'Net Profit': total_profit
})

print("\n" + "="*50)
print("FINANCIAL PERFORMANCE SUMMARY")
print("="*50)
print(fin_df.to_string(index=False, formatters={
    'Total Revenue': '${:,.2f}'.format,
    'Total Cost': '${:,.2f}'.format,
    'Net Profit': '${:,.2f}'.format
}))
print("="*50)
print(f"TOTAL CUMULATIVE REVENUE: ${total_rev.sum():,.2f}")
print(f"TOTAL CUMULATIVE COST: ${total_cost.sum():,.2f}")
print(f"TOTAL CUMULATIVE PROFIT: ${total_profit.sum():,.2f}")


FINANCIAL PERFORMANCE SUMMARY
 Week Total Revenue Total Cost  Net Profit
    1   $263,237.50  $8,372.00 $254,865.50
    2   $171,900.00 $94,860.50  $77,039.50
    3   $250,475.00 $33,737.50 $216,737.50
    4   $261,837.50 $67,441.50 $194,396.00
    5   $280,775.00  $3,316.50 $277,458.50
    6   $241,650.00 $71,237.75 $170,412.25
    7   $264,762.50 $32,801.50 $231,961.00
    8   $320,700.00 $58,200.00 $262,500.00
    9   $217,787.50 $63,247.25 $154,540.25
   10   $273,662.50 $32,056.75 $241,605.75
   11   $243,987.50 $49,264.75 $194,722.75
   12   $250,012.50 $49,935.50 $200,077.00
TOTAL CUMULATIVE REVENUE: $3,040,787.50
TOTAL CUMULATIVE COST: $564,471.50
TOTAL CUMULATIVE PROFIT: $2,476,316.00


In [21]:
cost_data = []
total_all_time = total_cost.sum()

categories = {
    'Production': lambda p, t: value(model.unit_production_cost[f'{p}_{t}'] * model.X[p, t]),
    'Labor (Reg)': lambda p, t: value(model.unit_reg_labor_cost[f'{p}_{t}'] * model.R[p, t]),
    'Labor (OT)': lambda p, t: value(model.unit_ot_labor_cost[f'{p}_{t}'] * model.O[p, t]),
    'Hiring/Firing': lambda p, t: value(model.hiring_cost[f'{p}_{t}'] * model.H[p, t] + model.firing_cost[f'{p}_{t}'] * model.F[p, t]),
    'Inventory': lambda p, t: sum(model.probs[s] * (value(model.unit_holding_cost[f'{p}_{t}']) * (value(model.I_regular[p, t, s]) + 3*value(model.I_overflow[p, t, s]))) for s in model.S),
    'Backorder': lambda p, t: sum(model.probs[s] * (value(model.unit_backorder_cost[f'{p}_{t}']) * value(model.B[p, t, s])) for s in model.S)
}

for cat, func in categories.items():
    cat_total = sum(func(p, t) for p in products_list for t in model.T)
    cost_data.append({
        'Category': cat,
        'Total Cost ($)': cat_total,
        'Percentage': (cat_total / total_all_time) * 100
    })

cost_df = pd.DataFrame(cost_data)
print("\n" + "="*50)
print("COST COMPOSITION ANALYSIS")
print("="*50)
print(cost_df.to_string(index=False, formatters={'Total Cost ($)': '{:,.2f}'.format, 'Percentage': '{:.2f}%'.format}))


COST COMPOSITION ANALYSIS
     Category Total Cost ($) Percentage
   Production      65,348.00     11.58%
  Labor (Reg)     447,353.00     79.25%
   Labor (OT)           0.00      0.00%
Hiring/Firing           0.00      0.00%
    Inventory      42,582.25      7.54%
    Backorder       9,188.25      1.63%


In [24]:
# --- Plotting ---
fig = make_subplots(
    rows=4, cols=1,
    shared_xaxes=True,
    subplot_titles=["TOTAL COMPANY FINANCIALS", "Product A Finance", "Product B Finance", "Product C Finance"],
    vertical_spacing=0.05
)

# Total Company
fig.add_trace(go.Bar(x=time_periods, y=total_rev, name='Total Revenue', marker_color='green', opacity=0.6), row=1, col=1)
fig.add_trace(go.Bar(x=time_periods, y=total_cost, name='Total Cost', marker_color='red', opacity=0.6), row=1, col=1)
fig.add_trace(go.Scatter(x=time_periods, y=total_profit, name='Total Profit/Loss', line=dict(color='black', width=3)), row=1, col=1)

# Per Product
for i, p in enumerate(products_list, start=2):
    fig.add_trace(go.Bar(x=time_periods, y=product_data[p]['rev'], name=f'{p} Revenue', marker_color='mediumseagreen', showlegend=False), row=i, col=1)
    fig.add_trace(go.Bar(x=time_periods, y=product_data[p]['cost'], name=f'{p} Cost', marker_color='indianred', showlegend=False), row=i, col=1)
    fig.add_trace(go.Scatter(x=time_periods, y=product_data[p]['profit'], name=f'{p} Profit', line=dict(color='black', dash='dash'), showlegend=False), row=i, col=1)

fig.update_layout(height=1200, title_text="Financial Performance Analysis", barmode='group', template='plotly_white')
fig.show()

## 4.1 Production Demand Inventory per Scenario

In [42]:
X_vals = {p: [value(model.X[p, t]) for t in time_periods] for p in products}

D_vals = {p: {s: [value(model.demand[p, t, s]) for t in time_periods] 
             for s in scenarios} for p in products}

I_vals = {p: {s: [value(model.I_regular[p, t, s]) + value(model.I_overflow[p, t, s]) for t in time_periods] 
             for s in scenarios} for p in products}

# Configuration
scenarios = ['low', 'med', 'high']
products = ['A', 'B', 'C']
# Color mapping for scenarios
colors = {'low': '#2ca02c', 'med': '#1f77b4', 'high': '#d62728'} 
# Line styles for variables
styles = {'Production': 'solid', 'Inventory': 'dash', 'Demand': 'dot'}

fig = make_subplots(
    rows=len(products), cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=[f"Product {p} Analysis" for p in products]
)

for row, p in enumerate(products, start=1):
    # 1. Production (Stage 1 Decision - Constant across all scenarios)
    fig.add_trace(
        go.Scatter(
            x=time_periods, y=X_vals[p],
            name=f'Production ({p})',
            line=dict(color='black', width=3, dash=styles['Production']),
            legendgroup='Production',
            showlegend=(row == 1)
        ),
        row=row, col=1
    )
    
    for s in scenarios:
        # 2. Demand (Scenario Input)
        fig.add_trace(
            go.Scatter(
                x=time_periods, y=D_vals[p][s],
                name=f'Demand ({s})',
                mode='lines+markers',
                line=dict(color=colors[s], width=1, dash=styles['Demand']),
                marker=dict(symbol='circle', size=5),
                legendgroup=f'Scenario_{s}',
                showlegend=(row == 1)
            ),
            row=row, col=1
        )
        
        # 3. Inventory (Scenario Result)
        fig.add_trace(
            go.Scatter(
                x=time_periods, y=I_vals[p][s],
                name=f'Inventory ({s})',
                line=dict(color=colors[s], width=2, dash=styles['Inventory']),
                legendgroup=f'Scenario_{s}',
                showlegend=(row == 1)
            ),
            row=row, col=1
        )

fig.update_layout(
    height=1000,
    width=1200,
    title_text="Multi-Scenario Tactical Analysis: Stage 1 vs. Stage 2",
    template="plotly_white",
    hovermode='x unified',
    legend=dict(
                tracegroupgap=15,
                orientation="h",
                yanchor="bottom",
                y=1.05,
                xanchor="center",
                x=0.7
            )
)

fig.update_xaxes(title_text="Week", row=len(products), col=1)
fig.update_yaxes(title_text="Units")

fig.show()

## 4.2 Expected Inventory, Backorders, Production, Total Cost

In [53]:
Space_Reg_Exp = {p: [sum(model.probs[s] * value(model.I_regular[p, t, s]) * value(model.space_factor[p]) 
                     for s in model.S) for t in model.T] for p in products_list}


Space_Ovr_Exp = {p: [sum(model.probs[s] * value(model.I_overflow[p, t, s]) * value(model.space_factor[p]) 
                     for s in model.S) for t in model.T] for p in products_list}

# Ensure D_exp_vals is calculated
D_exp_vals = {p: [sum(model.probs[s] * value(model.demand[p, t, s]) for s in model.S) for t in model.T] for p in products_list}

fig = make_subplots(specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]],
                    rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.07,
                    subplot_titles=[f"Product {p}: Stock vs. Warehouse Footprint" for p in products_list])

for i, p in enumerate(products_list, start=1):
    
    # --- Secondary Axis: Units (Lines/Dots) ---
    # Expected Demand (New)
    fig.add_trace(go.Scatter(x=time_periods, y=D_exp_vals[p], name="Expected Demand", 
                             mode='lines', line=dict(color='red', dash='dash', width=1.5),
                             legendgroup="Demand", showlegend=(i==1)), row=i, col=1, secondary_y=True)
    
    # Produced Units
    fig.add_trace(go.Scatter(x=time_periods, y=X_vals[p], name="Produced Units (X)", 
                             mode='lines', line=dict(color='blue', dash='dot', width=2),
                             legendgroup="Prod", showlegend=(i==1)), row=i, col=1, secondary_y=True)
    
    # Units in Stock (Med Scenario)
    fig.add_trace(go.Scatter(x=time_periods, y=I_vals[p]['med'], name="Inventory Units (Med Scenario)", 
                             mode='lines+markers', line=dict(color='black', width=1), marker=dict(size=4),
                             legendgroup="Stock", showlegend=(i==1)), row=i, col=1, secondary_y=True)
    
    # --- Primary Axis: Warehouse Space (Stacked Area) ---
    fig.add_trace(go.Scatter(x=time_periods, y=Space_Reg_Exp[p], name="Reg Space Used", stackgroup=f'stack{i}',
                             fillcolor='rgba(46, 204, 113, 0.4)', line=dict(width=0),
                             legendgroup="RegSpace", showlegend=(i==1)), row=i, col=1, secondary_y=False)
    
    fig.add_trace(go.Scatter(x=time_periods, y=Space_Ovr_Exp[p], name="Overflow Space Used", stackgroup=f'stack{i}',
                             fillcolor='rgba(231, 76, 60, 0.4)', line=dict(width=0),
                             legendgroup="OvrSpace", showlegend=(i==1)), row=i, col=1, secondary_y=False)

fig.update_layout(height=1000, width=1200, title_text="Inventory Strategy: Units vs. Space Footprint", 
                  template='plotly_white', hovermode='x unified',
                  legend=dict(
                        tracegroupgap=10,
                        orientation="h",
                        yanchor="bottom",
                        y=1.05,
                        xanchor="center",
                        x=1.05
                    ))

fig.update_yaxes(title_text="Pallet Spots (Area Used)", secondary_y=False)
fig.update_yaxes(title_text="Physical Units", secondary_y=True)
fig.update_xaxes(title_text="Week", row=3, col=1)

fig.show()

## 4.3 Workforce

In [57]:
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.06,
    specs=[[{"secondary_y": True}],
           [{"secondary_y": True}],
           [{"secondary_y": True}]],
    subplot_titles=[f"Product {p}: Labor Capacity, Usage, and Production" for p in products_list]
)

for i, p in enumerate(products_list, start=1):

    # Capacities
    reg_capacity = [value(model.W[p, t] * model.max_hours_per_worker) for t in model.T]
    max_capacity = [
        value(model.W[p, t] * model.max_hours_per_worker * (1 + model.over_time_fraction))
        for t in model.T
    ]

    # Usage
    R_hours = [value(model.R[p, t]) for t in model.T]
    O_hours = [value(model.O[p, t]) for t in model.T]

    # Required hours
    req_hours = [
        value(model.required_hours_per_unit[p] * model.X[p, t])
        for t in model.T
    ]

    # ---- Primary axis: hours ----

    fig.add_trace(
        go.Scatter(
            x=time_periods, y=max_capacity,
            name="Max Capacity (Reg + OT)",
            line=dict(dash='dash', color='gray'),
            showlegend=(i == 1)
        ),
        row=i, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=time_periods, y=reg_capacity,
            name="Regular Capacity",
            line=dict(color='royalblue'),
            showlegend=(i == 1)
        ),
        row=i, col=1
    )

    fig.add_trace(
        go.Bar(
            x=time_periods, y=req_hours,
            name="Required Hours",
            marker_color='orange',
            opacity=0.5,
            showlegend=(i == 1)
        ),
        row=i, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=time_periods, y=R_hours,
            name="Regular Hours Used",
            line=dict(color='green', width=2),
            showlegend=(i == 1)
        ),
        row=i, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=time_periods, y=O_hours,
            name="Overtime Hours Used",
            line=dict(color='red', width=2),
            showlegend=(i == 1)
        ),
        row=i, col=1
    )

    # ---- Secondary axis: production ----

    fig.add_trace(
        go.Scatter(
            x=time_periods, y=X_vals[p],
            name="Units Produced",
            line=dict(color='black', width=3),
            showlegend=(i == 1)
        ),
        row=i, col=1,
        secondary_y=True
    )

# Layout polish
fig.update_layout(
    height=1100,
    width=1200,
    title_text="Labor Capacity, Utilization, Overtime Stress, and Production Output",
    template="plotly_white",
    legend=dict(
                tracegroupgap=10,
                orientation="h",
                yanchor="bottom",
                y=1.05,
                xanchor="center",
                x=1.05
            )
)

fig.update_yaxes(title_text="Labor Hours", secondary_y=False)
fig.update_yaxes(title_text="Units Produced", secondary_y=True)

fig.show()


## 4.4 Two-stage Stochastic Optimization vs JIT Production Planning

In [38]:

batch_labor_reg = [sum(value(model.unit_reg_labor_cost[f'{p}_{t}'] * model.R[p, t]) for p in products_list) for t in model.T]
batch_labor_ot = [sum(value(model.unit_ot_labor_cost[f'{p}_{t}'] * model.O[p, t]) for p in products_list) for t in model.T]
batch_hr_costs = [sum(value(model.hiring_cost[f'{p}_{t}'] * model.H[p, t] + model.firing_cost[f'{p}_{t}'] * model.F[p, t]) for p in products_list) for t in model.T]


batch_prod_costs = [sum(value(model.unit_production_cost[f'{p}_{t}'] * model.X[p, t]) for p in products_list) for t in model.T]
batch_inv_reg = [sum(model.probs[s] * value(model.unit_holding_cost[f'{p}_{t}'] * model.I_regular[p, t, s]) for p in products_list for s in model.S) for t in model.T]
batch_inv_ovr = [sum(model.probs[s] * 3 * value(model.unit_holding_cost[f'{p}_{t}'] * model.I_overflow[p, t, s]) for p in products_list for s in model.S) for t in model.T]
batch_backorder = [sum(model.probs[s] * value(model.unit_backorder_cost[f'{p}_{t}'] * model.B[p, t, s]) for p in products_list for s in model.S) for t in model.T]


fig2 = go.Figure()

# --- Stacked Area: The Optimized Strategy (What you are actually doing) ---
fig2.add_trace(go.Scatter(x=time_periods, y=batch_labor_reg, name="Labor (Regular)", stackgroup='batch', fillcolor='rgba(100, 149, 237, 0.8)', line=dict(width=0)))
fig2.add_trace(go.Scatter(x=time_periods, y=batch_labor_ot, name="Labor (Overtime)", stackgroup='batch', fillcolor='rgba(255, 165, 0, 0.8)', line=dict(width=0)))
fig2.add_trace(go.Scatter(x=time_periods, y=batch_hr_costs, name="HR (Hire/Fire)", stackgroup='batch', fillcolor='rgba(128, 128, 128, 0.8)', line=dict(width=0)))
fig2.add_trace(go.Scatter(x=time_periods, y=batch_prod_costs, name="Production (Material)", stackgroup='batch', fillcolor='royalblue', line=dict(width=0)))
fig2.add_trace(go.Scatter(x=time_periods, y=batch_inv_reg, name="Reg Inv Cost", stackgroup='batch', fillcolor='mediumseagreen', line=dict(width=0)))
fig2.add_trace(go.Scatter(x=time_periods, y=batch_inv_ovr, name="Overflow Cost (3x)", stackgroup='batch', fillcolor='lightcoral', line=dict(width=0)))
fig2.add_trace(go.Scatter(x=time_periods, y=batch_backorder, name="Backorder Penalty", stackgroup='batch', fillcolor='crimson', line=dict(width=0)))


fig2.update_layout(
    title="Strategy Comparison: Batching Strategy vs. JIT Burdened Cost",
    xaxis_title="Week",
    yaxis_title="Expected Cost ($)",
    template='plotly_white',
    height=700,
    hovermode='x unified'
)
fig2.show()

## 4.5 Inventory Turnover and Average Unit Cost

In [29]:
import pandas as pd

health_data = []

for p in products_list:
    # 1. Total Expected Demand (across all scenarios and time periods)
    total_exp_demand = sum(model.probs[s] * value(model.demand[p, t, s]) 
                           for s in model.S for t in model.T)
    
    # 2. Total Units Produced and Sold
    total_produced = sum(value(model.X[p, t]) for t in model.T)
    total_demand_met = sum(model.probs[s] * (value(model.demand[p, t, s]) - value(model.B[p, t, s])) 
                           for s in model.S for t in model.T)
    
    # 3. Average Inventory Level
    avg_inv = sum(model.probs[s] * value(model.I_regular[p, t, s] + model.I_overflow[p, t, s]) 
                  for s in model.S for t in model.T) / len(time_periods)
    
    # 4. Inventory Turnover
    turnover = total_demand_met / avg_inv if avg_inv > 0 else float('inf')
    
    # 5. Total Burdened Cost (S1 + Expected S2)
    prod_cost = sum(value(model.unit_production_cost[f'{p}_{t}'] * model.X[p, t]) for t in model.T)
    labor_cost = sum(value(model.unit_reg_labor_cost[f'{p}_{t}'] * model.R[p, t] + 
                           model.unit_ot_labor_cost[f'{p}_{t}'] * model.O[p, t]) for t in model.T)
    hr_cost = sum(value(model.hiring_cost[f'{p}_{t}'] * model.H[p, t] + 
                        model.firing_cost[f'{p}_{t}'] * model.F[p, t]) for t in model.T)
    storage_penalty = sum(model.probs[s] * (
        value(model.unit_holding_cost[f'{p}_{t}']) * (value(model.I_regular[p, t, s]) + 3*value(model.I_overflow[p, t, s])) + 
        value(model.unit_backorder_cost[f'{p}_{t}']) * value(model.B[p, t, s])
    ) for s in model.S for t in model.T)
    
    total_burdened_cost = prod_cost + labor_cost + hr_cost + storage_penalty
    avg_unit_cost = total_burdened_cost / total_produced if total_produced > 0 else 0
    
    # 6. Margin %
    margin = (prices[p] - avg_unit_cost) / prices[p]
    
    health_data.append({
        'Product': p,
        'Total Demand': int(total_exp_demand),
        'Units Produced': int(total_produced),
        'Avg Inventory': round(avg_inv, 1),
        'Turnover Ratio': round(turnover, 2),
        'Avg Unit Cost ($)': round(avg_unit_cost, 2),
        'Profit Margin (%)': round(margin * 100, 1)
    })

health_df = pd.DataFrame(health_data)

print("\n" + "="*85)
print("FINAL PRODUCT HEALTH & OPERATIONAL EFFICIENCY")
print("="*85)
print(health_df.to_string(index=False))
print("="*85)


FINAL PRODUCT HEALTH & OPERATIONAL EFFICIENCY
Product  Total Demand  Units Produced  Avg Inventory  Turnover Ratio  Avg Unit Cost ($)  Profit Margin (%)
      A          2829            3045          313.8            8.56              51.09               65.9
      B          2552            2702          333.5            7.26              61.72               87.7
      C          2295            2464          327.7            6.70              98.27               84.9
